In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from snorkel.learning import GenerativeModel
from scipy import sparse
import matplotlib.pyplot as plt
from functools import partial
from sklearn.metrics import f1_score
from scipy.optimize import minimize


/anaconda2/envs/snorkel/lib/python3.6/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [2]:
# Generate Primitives
def has_bike(object_names):
    if ('cycle' in object_names) or ('bike' in object_names) or ('bicycle' in object_names):
        return 1
    else:
        return 0
def has_human(object_names):
    if (('person' in object_names) or ('woman' in object_names) or ('man' in object_names)) \
        and (('bicycle' in object_names) or 'bicycles' in object_names):
        return 1
    else:
        return 0
def has_road(object_names):
    if ('road' in object_names) or ('street' in object_names) or ('concrete' in object_names):
        return 1
    else:
        return 0
def has_cars(object_names):
    if ('car' in object_names) or ('cars' in object_names) or \
        ('bus' in object_names) or ('buses' in object_names) or \
        ('truck' in object_names) or ('trucks' in object_names):
        return 1
    else:
        return 0

In [3]:
from primitive_helpers import bike_human_distance, bike_human_size, bike_human_nums
from data_loader import DataLoader
loader = DataLoader()

def create_primitives(loader):
    m = 7 # number of primitives
    primitive_mtx = np.zeros((loader.train_num,m))

    for i in range(loader.train_num):
        primitive_mtx[i,0] = has_human(loader.train_object_names[i])
        primitive_mtx[i,1] = has_road(loader.train_object_names[i])
        primitive_mtx[i,2] = has_cars(loader.train_object_names[i])
        primitive_mtx[i,3] = has_bike(loader.train_object_names[i])

        primitive_mtx[i,4] = bike_human_distance(loader.train_object_names[i], 
                                                 loader.train_object_x[i], 
                                                 loader.train_object_y[i])

        area = np.multiply(loader.train_object_height[i], loader.train_object_width[i])
        primitive_mtx[i,5] = bike_human_size(loader.train_object_names[i], area)
        primitive_mtx[i,6] = bike_human_nums(loader.train_object_names[i])

    return primitive_mtx
primitive_mtx = create_primitives(loader)

p_keys = {
    'has_human': primitive_mtx[:,0],
    'has_road': primitive_mtx[:, 1],
    'has_cars': primitive_mtx[:, 2],
    'has_bike': primitive_mtx[:, 3],
    'bike_human_distance': primitive_mtx[:, 4],
    'bike_human_size': primitive_mtx[:, 5],
    'bike_human_num': primitive_mtx[:, 6]
   }
pos = list(np.where(loader.train_ground>0)[0])
neg = list(np.where(loader.train_ground<0)[0])[-len(pos):]
chosen_data = pos + neg
print(len(chosen_data))
loader.train_ground = loader.train_ground[chosen_data]
loader.train_num = len(chosen_data)

540


In [4]:
x1 = []
x2 = []
def LF_street(has_human, has_road):
    if has_human >= 1: 
        if has_road >= 1:
            return 1
        else:
            return 0
    return -1

# def LF_vehicles(has_human, has_cars):
#     if has_human >= 1: 
#         if has_cars >= 1:
#             return 1
#         else:
#             return 0
#     return -1

def LF_vehicles(has_human, has_bikes):
    if has_human >= 1: 
        if has_bikes >= 1:
            return 1
        else:
            return -1
    return -1

def LF_distance(has_human, has_bike, bike_human_distance, thre=8):
    if has_human >= 1:
        if has_bike >= 1: 
            x1.append(bike_human_distance)
            if bike_human_distance <= thre:
                return 1
            else:
                return 0
    else:
        return -1
    
def LF_size(has_human, has_bike, bike_human_size, thre=1000):
    if has_human >= 1:
        if has_bike >= 1: 
            x2.append(bike_human_size)
            if bike_human_size <= thre:
                return 1
            else:
                return 0
    else:
        return -1
    
    
def LF_number(has_human, has_bike, bike_human_num):
    if has_human >= 1:
        if has_bike >= 1: 
            if bike_human_num >= 2:
                return 1
            if bike_human_num >= 1:
                return 0
            if bike_human_num >= 0:
                return 1 
    else:
        return -1

In [5]:
def f(alpha, L_fns=None, ind=None, weights=None, epochs=None):
    """L_fns: list of functions
       l: list of indicator for whether thresholds involve in the particular function 
    """
    #alpha = np.exp(alpha)
    for i in range(len(ind)):
        L_fns[ind[i]] = partial(L_fns[ind[i]], thre=alpha[i])
        
    L = np.zeros((len(L_fns),loader.train_num)).astype(int)
    
    for j in range(len(chosen_data)):
        i = chosen_data[j]
        L[0,j] = L_fns[0](p_keys['has_human'][i], p_keys['has_road'][i])
        L[1,j] = L_fns[1](p_keys['has_human'][i], p_keys['has_cars'][i])
        L[2,j] = L_fns[2](p_keys['has_human'][i], p_keys['has_bike'][i], p_keys['bike_human_distance'][i])
        L[3,j] = L_fns[3](p_keys['has_human'][i], p_keys['has_bike'][i], p_keys['bike_human_size'][i])
        L[4,j] = L_fns[4](p_keys['has_human'][i], p_keys['has_bike'][i], p_keys['bike_human_num'][i])
        
    L_train = sparse.csr_matrix(L.T)
    gen_model = GenerativeModel()
    gen_model.train(L.T, epochs=epochs, decay=0.95, step_size= 0.01/ L.shape[1], reg_param=1e-6)
    if weights is not None:
        gen_model.weights = weights
    train_marginals, likelihood = gen_model.marginals(L_train)
    labels = 2 * (train_marginals > 0.5) - 1
    gen = np.mean(labels == loader.train_ground)
    f1_gen = f1_score(loader.train_ground, labels)
    if weights is None:
        return (gen,f1_gen), gen_model.weights, -likelihood 
    else:
        return -likelihood 

    
    

In [6]:
L_fns = [LF_street, LF_vehicles, LF_distance, LF_size, LF_number]
ind = [2,3]
alpha = np.ones((2,))*100
alpha = [20,5000]
# init = np.zeros((3,2))
# init[:,0] = [100, 200, 300]
# init[:,1] = [1, 10, 100]

iters = 100
for i in range(100):
    ans, weights, _ = f(alpha, L_fns=L_fns, ind=ind, epochs=500)
    f_new = partial(f, L_fns=L_fns, ind=ind, epochs=0, weights=weights)
    
    alpha = minimize(f_new, alpha, method='Nelder-Mead').x
    print(alpha)
    print(ans) 

# f_new = partial(f, L_fns=L_fns, ind=ind, epochs=0, weights=weights)
# init = np.zeros((3,2))
# init[:,0] = [1, 100, 1000]
# init[:,1] = [1, 100, 1000]
# a = minimize(f_new,alpha, options={'initial_simplex' :init}, method='Nelder-Mead')
    
    


[   21.  4750.]
(0.66666666666666663, 0.62500000000000011)
[   7526.71812744  182623.88839722]
(0.66666666666666663, 0.62500000000000011)
[   7526.71812744  182623.88839722]
(0.61111111111111116, 0.60227272727272729)
[   7526.71812744  182623.88839722]
(0.61111111111111116, 0.60227272727272729)
[   7526.71812744  182623.88839722]
(0.61111111111111116, 0.60227272727272729)
[   7526.71812744  182623.88839722]
(0.61111111111111116, 0.60227272727272729)
[   7526.71812744  182623.88839722]
(0.61111111111111116, 0.60227272727272729)


KeyboardInterrupt: 

In [ ]:
print(a)